<a href="https://colab.research.google.com/github/rkrisanoff/geographical-information-retrieval/blob/main/GIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install natasha
!pip install yandex-geocoder

In [93]:
from pprint import pprint

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [ ]:
import glob
from google.colab import drive
drive.mount('/content/drive')
news = glob.glob(f"/content/drive/MyDrive/GIR/FONTANKA-NEWS/little_news/*.txt")

In [ ]:
# import gdown
# Реальный датасет будет намного больше
# shared_link = 'https://drive.google.com/drive/folders/1YM3wLKR9qJg1SuTTk9UG2iISEObSNtw0?usp=share_link'
# output = 'news'
# news = gdown.download_folder(url=shared_link,output=output,remaining_ok=True,quiet=True) 

In [105]:
documents = dict()

for new_path in news:
  text = ""
  with open(new_path) as new:
    category,title,text = new.readlines()[:3]
    if title[-1:] == '\n':
      title = title[:-1]
  doc = Doc(text)
  doc.segment(segmenter) 
  doc.tag_morph(morph_tagger)
  for token in doc.tokens:
    token.lemmatize(morph_vocab)
  doc.tag_ner(ner_tagger)
  for span in doc.spans:
    span.normalize(morph_vocab)
    span.extract_fact(names_extractor)
    

  documents[title] = []
  for span in doc.spans:
    if span.type == "LOC":
      documents[title].append(span.normal)

In [ ]:
for title,words in list(documents.items()):
  print("")
  print(title)
  print('!'+'-'*(len(title)-2)+'!')
  print(words)
  print('^'+'-'*(len(title)-2)+'^')

Кластеризация (пока только начало)

In [108]:
from decimal import Decimal
from yandex_geocoder import Client
# api key
client = Client("")

In [109]:
texts = list(documents.values())
locations = dict()
undefined_locations = set()
for text in texts:
  for place in text:
    if place in undefined_locations:
      continue
    if place not in locations:
      try:
        coord = client.coordinates(place)
        locations[place] = float(coord[0]),float(coord[1])
      except:
        undefined_locations.add(place)

In [ ]:
undefined_locations

In [ ]:
locations